In [149]:
# imports
import re
import string
import time
import random
import requests
import regex as re
from bs4 import BeautifulSoup
import csv
import os
from os import path
import pandas as pd
from datetime import datetime, timezone

In [150]:
# Criar arquivo CSV
def baseArchive():
    basePath = os.path.join(os.getcwd(), 'db', 'dataFATO.csv')
    os.makedirs(os.path.dirname(basePath), exist_ok=True)
    if not os.path.exists(basePath):
        with open(basePath, 'w', encoding='utf-8') as baseArchive:
            baseArchive.write('titulo,resumo,URL\n')  # Cabeçalho compatível com os dados
    print(f"Arquivo base criado ou já existente em: {basePath}")
    return basePath


In [151]:
# limpeza
def cleanSentence(phrase):
    # Convertendo para minúsculas
    phrase = phrase.lower()

    # Removendo URLs
    phrase = re.sub(r'http\S+|www\S+|https\S+', '', phrase, flags=re.MULTILINE)

    # Removendo números
    phrase = re.sub(r'\d+', '', phrase)

    # Removendo pontuação
    phrase = phrase.translate(str.maketrans('', '', string.punctuation))

    # Removendo espaços extras
    phrase = phrase.strip()

    # Removendo múltiplos espaços
    phrase = re.sub(r'\s+', ' ', phrase)

    return phrase
    

In [152]:
def get_data(url_model):
    print(f"Obtendo artigo em: {url_model}")
    response = requests.get(url_model)
    if response.status_code != 200:
        print(f"Erro ao acessar curso em {url_model}: {response.status_code}")
        return []

    data_base_news = []
    
    base_url = "https://g1.globo.com/index/feed/pagina-{}.ghtml"
    for page_num in range(1, 250):  
        page_url = base_url.format(page_num)

        response = requests.get(page_url)
        if response.status_code != 200:
            print(f"Erro ao acessar curso em {page_url}: {response.status_code}")
            return []


        if response:

            soup = BeautifulSoup(response.content, 'html.parser')

            content_seccion = soup.find('div', class_='bastian-page')

            articles_seccion = content_seccion.find_all('div', class_='bastian-feed-item')

            for seccion in articles_seccion: 
                title_seccion = seccion.find('div', class_='_evt')
                title = title_seccion.find('p')
                title_text = title.text.strip()

                link = seccion.find('a')
                if link and link.has_attr('href'):
                    URL = link['href']
                else:
                    continue

                resumo = get_resume(URL)
                for resum in resumo:

                    data_base_news.append({
                        'titulo': title_text,
                        **resum,
                        'URL': URL
                    })
                    time.sleep(random.randrange(1, 2))                
            print(f'coletado! \nPÁGINA: {page_num}')
                
    return data_base_news



In [153]:
def get_resume(url):
    print(f"Obtendo resumo em: {url}")
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Erro ao acessar curso em {url}: {response.status_code}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    data_resume = []

    content_div = soup.find('div', class_='mc-article-header')
    resume_section = content_div.find('div', class_="medium-centered subtitle") if content_div else None
    resume_list = resume_section.find_next('h2') if resume_section else None

    if resume_section:
        resumo = resume_list.text.strip()
        data_resume.append({
            'resumo': resumo,
        })
        print(f'coletado, {resumo}')

    return data_resume


In [154]:
def saveData(data, filename=None):
    if filename is None:
        filename = baseArchive()

    df = pd.DataFrame(data)

    try:
        # Caso o arquivo já exista
        if os.path.exists(filename):
            existing_df = pd.read_csv(filename)
            
            # Combina os novos dados com os existentes, sem remover duplicatas
            combined_df = pd.concat([existing_df, df], ignore_index=True)
            combined_df.to_csv(filename, index=True, index_label='index')
        else:
            # Caso seja o primeiro salvamento
            df.to_csv(filename, index=True, index_label='index')

        print(f"Dados salvos no arquivo '{filename}' com sucesso!")

    except Exception as e:
        print(f"Erro ao salvar dados: {e}")

In [ ]:
if __name__ == "__main__":
    url_base = "https://g1.globo.com/"
    data = get_data(url_base)

    if data:
        saveData(data)
    else:
        print(f'nenhum dado encontrado')

Obtendo artigo em: https://g1.globo.com/
Obtendo resumo em: https://g1.globo.com/politica/noticia/2025/02/20/moraes-videos-e-audios-delacao-mauro-cid.ghtml
coletado, Transcrições já tinham sido tornadas públicas nesta quarta, mas mídias eram mantidas em sigilo. Material foi liberado nesta quinta e inclui horas de depoimentos no acordo de delação.
Obtendo resumo em: https://g1.globo.com/politica/blog/andreia-sadi/post/2025/02/20/ao-ter-prisao-decretada-video-mostra-que-mauro-cid-levou-as-maos-a-cabeca.ghtml
coletado, Nesta quinta-feira (20), Moraes tornou públicos os vídeos e áudios da delação de Mauro Cid.
Obtendo resumo em: https://g1.globo.com/politica/blog/andreia-sadi/post/2025/02/20/advogado-de-bolsonaro-diz-que-vai-pedir-anulacao-de-mauro-cid.ghtml
coletado, "O surpreendente é que eu tenha que pedir a anulação", diz Celso Vilardi. Trechos das falas de Cid foram usados em denúncia da PGR contra Bolsonaro por tentativa de golpe de estado.
Obtendo resumo em: https://g1.globo.com/pol